In [30]:
import numpy as np
import pandas as pd
import xarray as xr
import sys
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as datetime

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from torch.optim.lr_scheduler import StepLR
import torch.optim as optim
import pickle

from sklearn.preprocessing import MinMaxScaler

In [32]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [31]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_stacked_layers):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_stacked_layers = num_stacked_layers

        self.lstm = nn.LSTM(input_size, hidden_size, num_stacked_layers, 
                            batch_first=True) 
        
        self.fc = nn.Linear(hidden_size, 40*24)

    def forward(self, x):
        batch_size = x.size(0) # Get the batch size
        h0 = torch.zeros(self.num_stacked_layers, batch_size, self.hidden_size).to(device) # Initial hidden state
        c0 = torch.zeros(self.num_stacked_layers, batch_size, self.hidden_size).to(device) # Initial cell state
        
        out, _ = self.lstm(x, (h0, c0)) 
        out = self.fc(out[:, -1, :]) # Get the last output of the LSTM
        out = out.view(-1, 40, 24) # Reshape to (batch_size, mlat, mlt)
        return out
    
    def reset_states(self):
        # Reset the internal states of the LSTM layer
        self.lstm.reset_parameters()

# Instantiate the model with lookback size
model = LSTM(30, 64, 2)
model.load_state_dict(torch.load('lstm_ampere_id3.pt'))
model

LSTM(
  (lstm): LSTM(30, 64, num_layers=2, batch_first=True)
  (fc): Linear(in_features=64, out_features=960, bias=True)
)

In [6]:
def open_multiple_nc(nc_files):
    ds = xr.open_mfdataset(nc_files, concat_dim='nRec', combine='nested', parallel=True)

    #ds = ds.sel(nRec=slice(16500, 17000))

    return ds

dir = '/home/sachin/Documents/NIPR/Research/Data/AMPERE/processed/excluded/*.nc'
open_multi_amp = open_multiple_nc(dir)
open_multi_amp

<xarray.Dataset> Size: 7MB
Dimensions:         (nRec: 1, dt: 707, mlat: 50, mlt: 24)
Coordinates:
  * dt              (dt) datetime64[ns] 6kB 2013-05-14 ... 2013-05-14T23:58:00
  * mlat            (mlat) float64 400B 40.0 41.02 42.04 ... 87.96 88.98 90.0
  * mlt             (mlt) float64 192B 0.0 1.043 2.087 3.13 ... 21.91 22.96 24.0
Dimensions without coordinates: nRec
Data variables: (12/13)
    jPar            (nRec, dt, mlat, mlt) float64 7MB dask.array<chunksize=(1, 707, 50, 24), meta=np.ndarray>
    BX_GSE          (nRec, dt) float64 6kB dask.array<chunksize=(1, 707), meta=np.ndarray>
    BY_GSE          (nRec, dt) float64 6kB dask.array<chunksize=(1, 707), meta=np.ndarray>
    BZ_GSE          (nRec, dt) float64 6kB dask.array<chunksize=(1, 707), meta=np.ndarray>
    flow_speed      (nRec, dt) int32 3kB dask.array<chunksize=(1, 707), meta=np.ndarray>
    proton_density  (nRec, dt) float64 6kB dask.array<chunksize=(1, 707), meta=np.ndarray>
    ...              ...
    AU_INDEX        (nRec, dt) float64 6kB dask.array<chunksize=(1, 707), meta=np.ndarray>
    SYM_H           (nRec, dt) float64 6kB dask.array<chunksize=(1, 707), meta=np.ndarray>
    ASY_H           (nRec, dt) float64 6kB dask.array<chunksize=(1, 707), meta=np.ndarray>
    F10.7           (nRec, dt) int32 3kB dask.array<chunksize=(1, 707), meta=np.ndarray>
    Kp              (nRec, dt) float64 6kB dask.array<chunksize=(1, 707), meta=np.ndarray>
    doy             (nRec, dt) int32 3kB dask.array<chunksize=(1, 707), meta=np.ndarray>

In [26]:
def split_to_df(ds):
    ds = ds.copy()
    df = ds.to_dataframe().reset_index()

    df_2 = df.copy()

    df_2 = df_2.drop(columns=['dt', 'jPar','mlat','mlt','nRec','doy'])

    return df, df_2

df_raw, df_proc = split_to_df(open_multi_amp)
df_raw

,nRec,dt,mlat,mlt,jPar,BX_GSE,BY_GSE,BZ_GSE,flow_speed,proton_density,AL_INDEX,AU_INDEX,SYM_H,ASY_H,F10.7,Kp,doy
0,0,2013-05-14 00:00:00,40.0,0.000000,0.125046,3.055,2.650,2.445,370,8.720,-87.0,22.5,18.0,31.0,151,1.3,134
1,0,2013-05-14 00:00:00,40.0,1.043478,0.093376,3.055,2.650,2.445,370,8.720,-87.0,22.5,18.0,31.0,151,1.3,134
2,0,2013-05-14 00:00:00,40.0,2.086957,0.032481,3.055,2.650,2.445,370,8.720,-87.0,22.5,18.0,31.0,151,1.3,134
3,0,2013-05-14 00:00:00,40.0,3.130435,-0.024483,3.055,2.650,2.445,370,8.720,-87.0,22.5,18.0,31.0,151,1.3,134
4,0,2013-05-14 00:00:00,40.0,4.173913,-0.056095,3.055,2.650,2.445,370,8.720,-87.0,22.5,18.0,31.0,151,1.3,134
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
848395,0,2013-05-14 23:58:00,90.0,19.826087,0.070397,-5.815,2.175,5.700,332,10.905,-72.0,42.0,4.0,11.5,151,1.7,134
848396,0,2013-05-14 23:58:00,90.0,20.869565,0.081375,-5.815,2.175,5.700,332,10.905,-72.0,42.0,4.0,11.5,151,1.7,134
848397,0,2013-05-14 23:58:00,90.0,21.913043,0.048848,-5.815,2.175,5.700,332,10.905,-72.0,42.0,4.0,11.5,151,1.7,134
848398,0,2013-05-14 23:58:00,90.0,22.956522,-0.007801,-5.815,2.175,5.700,332,10.905,-72.0,42.0,4.0,11.5,151,1.7,134


In [27]:
with open('scaler_id3.pkl', 'rb') as f:
    scaler = pickle.load(f)

col_names = df_proc.columns
norm_arr = scaler.transform(df_proc)
df_proc = pd.DataFrame(norm_arr, columns=col_names)
df_proc

/home/sachin/miniconda3/envs/torch_env/lib/python3.12/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(


,BX_GSE,BY_GSE,BZ_GSE,flow_speed,proton_density,AL_INDEX,AU_INDEX,SYM_H,ASY_H,F10.7,Kp
0,0.523950,0.499831,0.542542,0.188920,0.119428,0.932584,0.144339,0.756391,0.086471,0.437838,0.156627
1,0.523950,0.499831,0.542542,0.188920,0.119428,0.932584,0.144339,0.756391,0.086471,0.437838,0.156627
2,0.523950,0.499831,0.542542,0.188920,0.119428,0.932584,0.144339,0.756391,0.086471,0.437838,0.156627
3,0.523950,0.499831,0.542542,0.188920,0.119428,0.932584,0.144339,0.756391,0.086471,0.437838,0.156627
4,0.523950,0.499831,0.542542,0.188920,0.119428,0.932584,0.144339,0.756391,0.086471,0.437838,0.156627
...,...,...,...,...,...,...,...,...,...,...,...
848395,0.389709,0.493401,0.584002,0.134943,0.149771,0.939912,0.155898,0.714286,0.032078,0.437838,0.204819
848396,0.389709,0.493401,0.584002,0.134943,0.149771,0.939912,0.155898,0.714286,0.032078,0.437838,0.204819
848397,0.389709,0.493401,0.584002,0.134943,0.149771,0.939912,0.155898,0.714286,0.032078,0.437838,0.204819
848398,0.389709,0.493401,0.584002,0.134943,0.149771,0.939912,0.155898,0.714286,0.032078,0.437838,0.204819


In [28]:
df_proc = df_proc.to_numpy()
df_proc = torch.tensor(df_proc, dtype=torch.float32)
df_proc

tensor([[0.5240, 0.4998, 0.5425,  ..., 0.0865, 0.4378, 0.1566],
        [0.5240, 0.4998, 0.5425,  ..., 0.0865, 0.4378, 0.1566],
        [0.5240, 0.4998, 0.5425,  ..., 0.0865, 0.4378, 0.1566],
        ...,
        [0.3897, 0.4934, 0.5840,  ..., 0.0321, 0.4378, 0.2048],
        [0.3897, 0.4934, 0.5840,  ..., 0.0321, 0.4378, 0.2048],
        [0.3897, 0.4934, 0.5840,  ..., 0.0321, 0.4378, 0.2048]])

In [46]:
df_proc.reshape(-1, 11, 30).shape
y = df_proc.reshape(-1, 11, 30)
#y = y.to(device)
y = model(y)
y

RuntimeError: Input and hidden tensors are not at the same device, found input tensor at cpu and hidden tensor at cuda:0

In [39]:
batch_size = 70
n = df_proc.size(0)
predictions = []

for i in range(0, n, batch_size):
    x = df_proc[i:i+batch_size].view(-1, 40, 30).to(device)
    print(x.shape)
    break
    #y = model(x)
    #redictions.append(y)

predictions = torch.cat(predictions)

RuntimeError: shape '[-1, 40, 30]' is invalid for input of size 770